In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
# from shapely.ops import split, snap, nearest_points
# import netana as na
import networkx as nx

# TOPOLOGIA 
- SPLIT LINES -- complete
- MERGE LINES
- OVERKILL OVERLAPING LINES
- CLIP LINES BY POLYGON
- DELETE ISLANDS (by id of larger dataset)

In [63]:
network_crs = "3857"
df_streets = gpd.read_file("./notebook/street_resistencia_corrientes_v3.shp")

unary_union_result = df_streets.unary_union.intersection(df_streets.unary_union)
gpd_lines = gpd.GeoDataFrame([unary_union_result], columns=['geometry'], geometry='geometry').explode(index_parts=False).reset_index(drop=True)

In [115]:
gpd_lines.crs = df_streets.crs
gpd_lines = gpd_lines.to_crs(network_crs)
gpd_lines.crs 

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [116]:
# calculate weights (TO DO)
gpd_lines['weight'] = gpd_lines.length

# Create LINES and POINTS DATASETS; and GRAPH

In [217]:
# create nodes and edges datasets
def create_nodes_and_edges_gdf(gpd_lines):
    # create lines
    # get starting and ending points
    gpd_lines['nd_st_coords'] = gpd_lines.apply(lambda row: Point(row.geometry.coords[0]), axis=1)
    gpd_lines['nd_en_coords'] = gpd_lines.apply(lambda row: Point(row.geometry.coords[1]), axis=1)
    gpd_lines['id'] = "e_"+ (gpd_lines.index+1).astype(str)

    # create points from lines
    # list all unique points from lines
    points_list = list(gpd_lines.nd_st_coords.unique()) + list(gpd_lines.nd_en_coords.unique())
    points_list = set(points_list)

    # create a point dataset
    gdf_points = pd.DataFrame(points_list, columns=['geometry'])
    gdf_points['node_id'] = "n_"+ (gdf_points.index+1).astype(str)
    gdf_points = gdf_points[['node_id', 'geometry']]

        # add points ids to lines dataset
    gpd_lines['nd_st'] = gpd_lines.apply(lambda row: list(gdf_points.loc[gdf_points.geometry == row.nd_st_coords, 'node_id'])[0], axis=1)
    gpd_lines['nd_en'] = gpd_lines.apply(lambda row: list(gdf_points.loc[gdf_points.geometry == row.nd_en_coords, 'node_id'])[0], axis=1)
    gpd_lines = gpd_lines[['id', 'nd_st', 'nd_en', 'weight', 'geometry']]

    return [gdf_points, gpd_lines]

# create graph, point, edges and mapping dict. from gdf
def graph_from_gdf(gdf_network, f_weight='weight', v_label=None, v_make_directed=False, v_directed=False, remove_isolated=False, remove_selfloop=False):

    gdf_nodes, gdf_edges = create_nodes_and_edges_gdf(gdf_network)
    gdf_nodes = gpd.GeoDataFrame(gdf_nodes)
    gdf_edges = gpd.GeoDataFrame(gdf_edges)
    gdf_nodes.crs = gdf_network.crs
    gdf_edges.crs = gdf_network.crs

    edgelist = list(gdf_edges.apply(lambda row: ( row.nd_st, row.nd_en, {'id':row.id, 'weight': row.weight}), axis=1))
    graph = nx.from_edgelist(edgelist)

    if remove_isolated == True:
        graph.remove_nodes_from(list(nx.isolates(graph)))
    if remove_selfloop == True:
        graph.remove_edges_from(nx.selfloop_edges(graph))

    if v_make_directed == True:
        graph = graph.to_directed(as_view=False)
    
    return [graph, gdf_nodes, gdf_edges]

In [218]:
G, gdf_nodes, gdf_edges  = graph_from_gdf(gpd_lines)
G

# SOURCES

In [219]:
# create a clean sources gdf
def create_sources_from_gdf(gdf_sources, v_crs_proj):
    
    gdf_sources_2 = gdf_sources.to_crs(v_crs_proj)
    gdf_sources_2 = gdf_sources_2.explode(index_parts=True).reset_index(drop=True)

    if 'id' in gdf_sources_2.columns:
        gdf_sources_2.drop(columns=['id'], inplace=True)
    gdf_sources_2['id'] = list(gdf_sources_2.index)
    gdf_sources_2['id'] = gdf_sources_2.apply(lambda row: "loc_"+str(row.id), axis=1) 

    gdf_sources_2 = gdf_sources_2[['id', 'geometry']]

    return gdf_sources_2

In [220]:
sources_file_path = "./notebook/source_corrientes.shp"

gdf_locations = gpd.read_file(sources_file_path)
gdf_locations = create_sources_from_gdf(gdf_locations, network_crs)
gdf_locations = gdf_locations.to_crs(network_crs)
print(gdf_locations.crs)
gdf_locations.head()

EPSG:3857


,id,geometry
0,loc_0,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,loc_1,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


In [221]:
""" nodes_in = gdf_nodes.sjoin(gpd.GeoDataFrame(geometry=gdf_locations.buffer(30)))
nodes_in = nodes_in.drop_duplicates(subset='id').reset_index(drop=True)
node_list = list(nodes_in.id)
node_list """

" nodes_in = gdf_nodes.sjoin(gpd.GeoDataFrame(geometry=gdf_locations.buffer(30)))\nnodes_in = nodes_in.drop_duplicates(subset='id').reset_index(drop=True)\nnode_list = list(nodes_in.id)\nnode_list "

In [222]:
def closest_nodes_to_sources(gdf_sources, gdf_nodes):
    gdf_amenities = gpd.sjoin_nearest(gdf_sources, gdf_nodes, how='inner')
    #gdf_amenities.rename(columns={'id_left':'id', 'id_right':'node_id'}, inplace=True)
    gdf_amenities = gdf_amenities[['id', 'node_id', 'geometry']]
    return gdf_amenities

gdf_locations = closest_nodes_to_sources(gdf_locations, gdf_nodes)
gdf_locations

,id,node_id,geometry
0,loc_0,n_5,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,loc_1,n_58,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


In [223]:
from netana.tools.calc_tools import split_line_by_point

def calculate_travel_time_to_all_nodes(graph, f_weight, loc_node_list):
    result = nx.multi_source_dijkstra_path_length(graph, 
                                        sources = loc_node_list, 
                                        cutoff=None, 
                                        weight=f_weight)
    df = pd.DataFrame(result.items(), columns=['node_id', 'weight_to'])
    return df

def asign_weight_values_to_nodes_and_edges(gdf_nodes, gdf_edges, df_weight):
    df_nodes= gdf_nodes.merge(df_weight[['node_id', 'weight_to']], on='node_id', how='left')
    df_nodes= df_nodes[['node_id', 'geometry', 'weight_to']]

    df_edges = gdf_edges.merge(df_weight[['node_id', 'weight_to']], left_on='nd_st', right_on='node_id', how='left')
    df_edges = df_edges.merge(df_weight[['node_id', 'weight_to']], left_on='nd_en', right_on='node_id', how='left')
    df_edges.rename(columns={'weight_to_x':'weight_start', 'weight_to_y':'weight_end'}, inplace=True)
    df_edges.drop(columns=['node_id_x', 'node_id_y'], inplace=True)

    return df_nodes, df_edges

def create_service_area_by_weight_limit(gdf_edges, weight_limit):

    gdf_edges['cat_limit'] = 'out'
    gdf_edges.loc[(gdf_edges.weight_start < weight_limit) & (gdf_edges.weight_end < weight_limit), 'cat_limit'] = 'ok'
    df_edges_ok = gdf_edges.loc[gdf_edges.cat_limit == 'ok'].reset_index(drop=True)
   
    df_edges__from_start = gdf_edges[(gdf_edges.weight_start < weight_limit) & (gdf_edges.weight_end > weight_limit)].reset_index(drop=True)
    if len(df_edges__from_start) > 0:
        df_edges__from_start['cat_limit'] = 'start'
        df_edges__from_start['weight_delta'] = (weight_limit - df_edges__from_start['weight_start']) / (df_edges__from_start['weight_end'] - df_edges__from_start['weight_start'])
        df_edges__from_start['geometry'] = df_edges__from_start.apply(lambda row: split_line_by_point(row.geometry, row.weight_delta, 'start'), axis=1)

    df_edges_from_end = gdf_edges[(gdf_edges.weight_start > weight_limit) & (gdf_edges.weight_end < weight_limit)].reset_index(drop=True)
    if len(df_edges_from_end) > 0:
        df_edges_from_end['cat_limit'] = 'end'
        df_edges_from_end['weight_delta'] = (weight_limit - df_edges_from_end['weight_end']) / (df_edges_from_end['weight_start'] - df_edges_from_end['weight_end'])
        df_edges_from_end['geometry'] = df_edges_from_end.apply(lambda row: split_line_by_point(row.geometry, row.weight_delta, 'end'), axis=1)

    df_edges_weight_limit = pd.concat([df_edges_ok, df_edges__from_start, df_edges_from_end])

    df_edges_weight_limit.drop(columns=['cat_limit', 'weight_delta'], inplace=True)
    df_edges_weight_limit['weight_limit'] = weight_limit
    return df_edges_weight_limit

def calculate_service_edges(graph, gdf_nodes, gdf_edges, f_weight, v_weight_limit, gdf_locations):
    list_source_nodes = list(gdf_locations['node_id'])
    df_weight_times = calculate_travel_time_to_all_nodes(graph, f_weight, list_source_nodes)
    gdf_nodes, gdf_edges = asign_weight_values_to_nodes_and_edges(gdf_nodes, gdf_edges, df_weight_times)
    df_edges_weight_limit = create_service_area_by_weight_limit(gdf_edges, v_weight_limit)
    return df_edges_weight_limit

In [227]:
df_edges_cost_limit_5_min = calculate_service_edges(G, gdf_nodes, gdf_edges, 'weight', 150, gdf_locations)
df_edges_cost_limit_5_min.to_file("./notebook/zz_edges-5min.shp")
df_edges_cost_limit_5_min.head()

C:\Users\Compumar\AppData\Local\Temp\ipykernel_16300\3682175214.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_edges_cost_limit_5_min.to_file("./notebook/zz_edges-5min.shp")


,id,nd_st,nd_en,weight,geometry,weight_start,weight_end,weight_limit
0,e_19,n_21,n_17,11.859794,"LINESTRING (-6546767.058 -3184186.989, -654677...",119.307678,107.447884,150
1,e_20,n_17,n_5,107.447884,"LINESTRING (-6546778.279 -3184183.149, -654688...",107.447884,0.000000,150
2,e_33,n_5,n_19,147.965708,"LINESTRING (-6546884.300 -3184165.694, -654685...",0.000000,147.965708,150
3,e_38,n_48,n_23,13.488133,"LINESTRING (-6546458.469 -3183690.815, -654646...",149.138383,135.650251,150
4,e_39,n_23,n_33,12.025383,"LINESTRING (-6546464.637 -3183702.811, -654646...",135.650251,147.675633,150


In [228]:
from netana.service_analysis.service_area_analysis import create_service_area
gdf_buffer = create_service_area(df_edges_cost_limit_5_min, 5, 100)
gdf_buffer.to_file("./notebook/zz_buffer.shp")
gdf_buffer

C:\Users\Compumar\AppData\Local\Temp\ipykernel_16300\2776910315.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_buffer.to_file("./notebook/zz_buffer.shp")


,geometry,id,nd_st,nd_en,weight,weight_start,weight_end,weight_limit
0,"MULTIPOLYGON (((-6546721.650 -3183661.908, -65...",e_19,n_21,n_17,11.859794,119.307678,107.447884,150


,geometry,id,nd_st,nd_en,weight,weight_start,weight_end,weight_limit
0,"MULTIPOLYGON (((-6546721.650 -3183661.908, -65...",e_19,n_21,n_17,11.859794,119.307678,107.447884,150
